In [25]:
# Import libraries
import pandas as pd

In [26]:
# Read data file
src_df = pd.read_csv('./data/dr_min_data(4).csv')
src_df.head()

,patient_id,age,gender,race,admission_date,diagnosis1,diagnosis2,diagnosis3,diagnosis4,diagnosis5,hcpcs,procedure1,procedure2,procedure3,procedure4,procedure5
0,P302,70,F,Unknown,12/14/2030,41071,51881,99672,78551,5856,81001,66,3607,45,40,3722
1,P430,62,F,Unknown,4/28/2021,41401,5849,4139,4148,V4582,82550,66,3607,3722,8853,45
2,P492,78,F,Unknown,7/7/2048,41011,42821,5185,4271,9971,82948,66,3606,3722,8853,45
3,P651,80,F,Unknown,2/18/2016,2848,42731,486,11284,5781,86901,4131,3893,4513,9904,8871
4,P679,68,F,Unknown,10/21/2013,9961,25040,40391,5856,41400,85014,3927,3895,3995,9904,3895


In [27]:
src_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   patient_id      1000 non-null   object
 1   age             1000 non-null   int64 
 2   gender          1000 non-null   object
 3   race            1000 non-null   object
 4   admission_date  1000 non-null   object
 5   diagnosis1      1000 non-null   object
 6   diagnosis2      1000 non-null   object
 7   diagnosis3      1000 non-null   object
 8   diagnosis4      1000 non-null   object
 9   diagnosis5      1000 non-null   object
 10  hcpcs           1000 non-null   object
 11  procedure1      1000 non-null   object
 12  procedure2      1000 non-null   object
 13  procedure3      1000 non-null   object
 14  procedure4      1000 non-null   object
 15  procedure5      1000 non-null   object
dtypes: int64(1), object(15)
memory usage: 125.1+ KB


In [28]:
# Flatten src_df to make querying and joining simpler

# Melting the DataFrame to long format
flat_src_df = pd.melt(src_df, id_vars=['patient_id', 'age', 'gender', 'race', 'admission_date'], 
                      var_name='record_info', value_name='CODE')

# Extracting record_type and record_num
flat_src_df['SAB'] = flat_src_df['record_info'].str.extract('(diagnosis|hcpcs|procedure)')
flat_src_df['record_num'] = flat_src_df['record_info'].str.extract('(\d+)', expand=False)

# Replacing NaN in record_num with 1 for hcpcs as it only occurs once
flat_src_df['record_num'].fillna('1', inplace=True)

# Dropping the original 'record_info' column as it's no longer needed
flat_src_df.drop('record_info', axis=1, inplace=True)

# Sorting the DataFrame based on patient_id and record_type for better readability
flat_src_df.sort_values(by=['patient_id', 'SAB', 'record_num'], inplace=True)

# Replace values of record_type to align with vocabulary
flat_src_df['SAB'] = flat_src_df['SAB'].replace({
    'diagnosis': 'ICD9CM',
    'hcpcs': 'HCPCS',
    'procedure': 'CPT'
})

# Resetting index of the DataFrame
flat_src_df.reset_index(drop=True, inplace=True)

In [29]:
flat_src_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   patient_id      11000 non-null  object
 1   age             11000 non-null  int64 
 2   gender          11000 non-null  object
 3   race            11000 non-null  object
 4   admission_date  11000 non-null  object
 5   CODE            11000 non-null  object
 6   SAB             11000 non-null  object
 7   record_num      11000 non-null  object
dtypes: int64(1), object(7)
memory usage: 687.6+ KB


In [30]:
flat_src_df.head(20)

,patient_id,age,gender,race,admission_date,CODE,SAB,record_num
0,P1,44,F,Black,3/8/2050,73005,ICD9CM,1
1,P1,44,F,Black,3/8/2050,3441,ICD9CM,2
2,P1,44,F,Black,3/8/2050,25000,ICD9CM,3
3,P1,44,F,Black,3/8/2050,2859,ICD9CM,4
4,P1,44,F,Black,3/8/2050,311,ICD9CM,5
5,P1,44,F,Black,3/8/2050,85027,HCPCS,1
6,P1,44,F,Black,3/8/2050,7765,CPT,1
7,P1,44,F,Black,3/8/2050,8622,CPT,2
8,P1,44,F,Black,3/8/2050,3893,CPT,3
9,P1,44,F,Black,3/8/2050,8622,CPT,4


In [31]:
# Read UMLS subset file
umls_df = pd.read_csv('C:/UMLS Files/umls-2023AB-full/2023AB-full/2023AB/META/MRCONSO.RRF', sep = '|', header= None)

# rename column names based on documenation (https://www.ncbi.nlm.nih.gov/books/NBK9685/)
umls_df.columns = [
    'CUI',
    'LAT',
    'TS',
    'LUI',
    'STT',
    'SUI',
    'ISPREF',
    'AUI',
    'SAUI',
    'SCUI',
    'SDUI',
    'SAB',
    'TTY',
    'CODE',
    'STR',
    'SRL',
    'SUPPRESS',
    'CVF',
    'UNKNOWN'] # This is not part of documentation, used UNKNOWN

C:\Users\vladc\AppData\Local\Temp\ipykernel_5244\1067810824.py:2: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  umls_df = pd.read_csv('C:/UMLS Files/umls-2023AB-full/2023AB-full/2023AB/META/MRCONSO.RRF', sep = '|', header= None)


In [32]:
umls_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15179540 entries, 0 to 15179539
Data columns (total 19 columns):
 #   Column    Dtype  
---  ------    -----  
 0   CUI       object 
 1   LAT       object 
 2   TS        object 
 3   LUI       object 
 4   STT       object 
 5   SUI       object 
 6   ISPREF    object 
 7   AUI       object 
 8   SAUI      float64
 9   SCUI      object 
 10  SDUI      object 
 11  SAB       object 
 12  TTY       object 
 13  CODE      object 
 14  STR       object 
 15  SRL       int64  
 16  SUPPRESS  object 
 17  CVF       float64
 18  UNKNOWN   float64
dtypes: float64(3), int64(1), object(15)
memory usage: 2.1+ GB


In [33]:
umls_df.head()

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,UNKNOWN
0,C0000005,ENG,P,L0000005,PF,S0007492,Y,A26634265,NaN,M0019694,D012711,MSH,PEP,D012711,(131)I-Macroaggregated Albumin,0,N,256.0,NaN
1,C0000005,ENG,S,L0270109,PF,S0007491,Y,A26634266,NaN,M0019694,D012711,MSH,ET,D012711,(131)I-MAA,0,N,256.0,NaN
2,C0000005,FRE,P,L6220710,PF,S7133957,Y,A13433185,NaN,M0019694,D012711,MSHFRE,PEP,D012711,Macroagrégats d'albumine marquée à l'iode 131,3,N,NaN,NaN
3,C0000005,FRE,S,L6215648,PF,S7133916,Y,A27488794,NaN,M0019694,D012711,MSHFRE,ET,D012711,MAA-I 131,3,N,NaN,NaN
4,C0000005,FRE,S,L6215656,PF,S7133956,Y,A27614225,NaN,M0019694,D012711,MSHFRE,ET,D012711,Macroagrégats d'albumine humaine marquée à l'i...,3,N,NaN,NaN


In [34]:
# Create mapping file
# Retain only columns that we need
cols = ['CUI', 'SAB', 'CODE', 'STR']
mapping_df = umls_df[cols]

# Retain only rows where Source / SAB is ICD9CM, HCPCS, or CPT and TTY is PT (Preferred Name)
mapping_df = mapping_df[((mapping_df['SAB'] == 'ICD9CM') | (mapping_df['SAB'] == 'CPT') | (mapping_df['SAB'] == 'HCPCS') | (mapping_df['SAB'] == 'SNOMEDCT_US'))]
mapping_df

,CUI,SAB,CODE,STR
19,C0000039,SNOMEDCT_US,102735002,Dipalmitoylphosphatidylcholine
26,C0000039,SNOMEDCT_US,102735002,Dipalmitoylphosphatidylcholine (substance)
61,C0000052,SNOMEDCT_US,58488005,"1,4-alpha-Glucan branching enzyme"
69,C0000052,SNOMEDCT_US,58488005,Branching enzyme
77,C0000052,SNOMEDCT_US,58488005,"Amylo-(1,4,6)-transglycosylase"
...,...,...,...,...
15179209,C5847626,SNOMEDCT_US,721271000124105,Non-interpretable minimum inhibitory concentra...
15179210,C5847626,SNOMEDCT_US,721271000124105,Non-interpretable MIC - minimum inhibitory con...
15179211,C5847626,SNOMEDCT_US,721271000124105,Non-interpretable minimum inhibitory concentra...
15179212,C5847627,SNOMEDCT_US,6011000124106,SNOMEDCT_US_2023_09_01 to ICD10CM_2023 Mappings


In [35]:
# Create 2 versions of the mapping dataframe one for getting CUI and another for getting SNOMED CT codes
mapping_df_CUI = mapping_df[((mapping_df['SAB'] == 'ICD9CM') | (mapping_df['SAB'] == 'CPT') | (mapping_df['SAB'] == 'HCPCS'))]
mapping_df_CUI

,CUI,SAB,CODE,STR
5961,C0000737,ICD9CM,789.0,Abdominal pain
5975,C0000737,ICD9CM,789.00,"Abdominal pain, unspecified site"
5998,C0000737,ICD9CM,789.00,Abdmnal pain unspcf site
6795,C0000768,ICD9CM,740-759.99,CONGENITAL ANOMALIES
6796,C0000768,ICD9CM,759.9,Congenital anomaly NOS
...,...,...,...,...
15044790,C5779936,ICD9CM,445.89,"Atheroembolism, site NEC"
15044873,C5779951,ICD9CM,173.09,Malig neo skin lip NEC
15044874,C5779951,ICD9CM,173.09,Other specified malignant neoplasm of skin of lip
15045163,C5780018,HCPCS,C9450,"Injection, fluocinolone acetonide intravitreal..."


In [36]:
mapping_df_SNOMED = mapping_df[mapping_df['SAB'] == 'SNOMEDCT_US']
mapping_df_SNOMED

,CUI,SAB,CODE,STR
19,C0000039,SNOMEDCT_US,102735002,Dipalmitoylphosphatidylcholine
26,C0000039,SNOMEDCT_US,102735002,Dipalmitoylphosphatidylcholine (substance)
61,C0000052,SNOMEDCT_US,58488005,"1,4-alpha-Glucan branching enzyme"
69,C0000052,SNOMEDCT_US,58488005,Branching enzyme
77,C0000052,SNOMEDCT_US,58488005,"Amylo-(1,4,6)-transglycosylase"
...,...,...,...,...
15179209,C5847626,SNOMEDCT_US,721271000124105,Non-interpretable minimum inhibitory concentra...
15179210,C5847626,SNOMEDCT_US,721271000124105,Non-interpretable MIC - minimum inhibitory con...
15179211,C5847626,SNOMEDCT_US,721271000124105,Non-interpretable minimum inhibitory concentra...
15179212,C5847627,SNOMEDCT_US,6011000124106,SNOMEDCT_US_2023_09_01 to ICD10CM_2023 Mappings


In [37]:
# Data prep of mapping file

# Retain only needed columns from mapping_df for the table join
mapping_df_CUI = mapping_df_CUI[['SAB', 'CODE', 'CUI']]
mapping_df_SNOMED = mapping_df_SNOMED[['CODE', 'CUI']]

# For ICD9CM remove the period
mapping_df_CUI.loc[mapping_df_CUI['SAB'] == 'ICD9CM', 'CODE'] = mapping_df_CUI.loc[mapping_df_CUI['SAB'] == 'ICD9CM', 'CODE'].str.replace('.', '', regex=False)

# Drop duplicates
mapping_df_CUI = mapping_df_CUI.drop_duplicates(subset=['SAB', 'CODE'], keep='first')
mapping_df_SNOMED = mapping_df_SNOMED.drop_duplicates(subset=['CUI'], keep='first')

mapping_df_CUI

C:\Users\vladc\AppData\Local\Temp\ipykernel_5244\1543162074.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapping_df_CUI.loc[mapping_df_CUI['SAB'] == 'ICD9CM', 'CODE'] = mapping_df_CUI.loc[mapping_df_CUI['SAB'] == 'ICD9CM', 'CODE'].str.replace('.', '', regex=False)


,SAB,CODE,CUI
5961,ICD9CM,7890,C0000737
5975,ICD9CM,78900,C0000737
6795,ICD9CM,740-75999,C0000768
6796,ICD9CM,7599,C0000768
7203,ICD9CM,5202,C0000770
...,...,...,...
15043654,ICD9CM,99586,C5779784
15044207,ICD9CM,06640,C5779840
15044789,ICD9CM,44589,C5779936
15044873,ICD9CM,17309,C5779951


In [38]:
mapping_df_SNOMED

,CODE,CUI
19,102735002,C0000039
61,58488005,C0000052
242,285407008,C0000097
322,13579002,C0000102
531,112116001,C0000163
...,...,...
15179202,721221000124109,C5847624
15179205,721261000124103,C5847625
15179209,721271000124105,C5847626
15179212,6011000124106,C5847627


In [39]:
# In SNOMED CT mapping dataframe change column name of CODE to SNOMED_CT_CODE
mapping_df_SNOMED = mapping_df_SNOMED.rename(columns={'CODE': 'SNOMED_CT_CODE'})
mapping_df_SNOMED

,SNOMED_CT_CODE,CUI
19,102735002,C0000039
61,58488005,C0000052
242,285407008,C0000097
322,13579002,C0000102
531,112116001,C0000163
...,...,...
15179202,721221000124109,C5847624
15179205,721261000124103,C5847625
15179209,721271000124105,C5847626
15179212,6011000124106,C5847627


In [40]:
# Data prep of source file
# For CPT code should have 5 characters, if not add leading zeroes
flat_src_df.loc[flat_src_df['SAB'] == 'CPT', 'CODE'] = flat_src_df.loc[flat_src_df['SAB'] == 'CPT', 'CODE'].apply(lambda x: x.zfill(5))

flat_src_df

,patient_id,age,gender,race,admission_date,CODE,SAB,record_num
0,P1,44,F,Black,3/8/2050,73005,ICD9CM,1
1,P1,44,F,Black,3/8/2050,3441,ICD9CM,2
2,P1,44,F,Black,3/8/2050,25000,ICD9CM,3
3,P1,44,F,Black,3/8/2050,2859,ICD9CM,4
4,P1,44,F,Black,3/8/2050,311,ICD9CM,5
...,...,...,...,...,...,...,...,...
10995,P999,84,F,White,4/29/2008,02263,CPT,1
10996,P999,84,F,White,4/29/2008,02263,CPT,2
10997,P999,84,F,White,4/29/2008,00222,CPT,3
10998,P999,84,F,White,4/29/2008,00222,CPT,4


In [41]:
# Use merge to left join tables and map the CUI
mapped_df_SNOMED = pd.merge(flat_src_df, mapping_df_CUI, how='left', left_on=['SAB', 'CODE'], right_on=['SAB', 'CODE'])

mapped_df_SNOMED

,patient_id,age,gender,race,admission_date,CODE,SAB,record_num,CUI
0,P1,44,F,Black,3/8/2050,73005,ICD9CM,1,C1963552
1,P1,44,F,Black,3/8/2050,3441,ICD9CM,2,C0030486
2,P1,44,F,Black,3/8/2050,25000,ICD9CM,3,C0375113
3,P1,44,F,Black,3/8/2050,2859,ICD9CM,4,C0002871
4,P1,44,F,Black,3/8/2050,311,ICD9CM,5,C0039492
...,...,...,...,...,...,...,...,...,...
10995,P999,84,F,White,4/29/2008,02263,CPT,1,NaN
10996,P999,84,F,White,4/29/2008,02263,CPT,2,NaN
10997,P999,84,F,White,4/29/2008,00222,CPT,3,C0370256
10998,P999,84,F,White,4/29/2008,00222,CPT,4,C0370256


In [43]:
# Use merge to left join tables SNOMED code given CUI
mapped_df_SNOMED = pd.merge(mapped_df_SNOMED , mapping_df_SNOMED, how='left', left_on=['CUI'], right_on=['CUI'])

mapped_df_SNOMED

,patient_id,age,gender,race,admission_date,CODE,SAB,record_num,CUI,SNOMED_CT_CODE
0,P1,44,F,Black,3/8/2050,73005,ICD9CM,1,C1963552,268016009
1,P1,44,F,Black,3/8/2050,3441,ICD9CM,2,C0030486,155031004
2,P1,44,F,Black,3/8/2050,25000,ICD9CM,3,C0375113,NaN
3,P1,44,F,Black,3/8/2050,2859,ICD9CM,4,C0002871,64593003
4,P1,44,F,Black,3/8/2050,311,ICD9CM,5,C0039492,173068002
...,...,...,...,...,...,...,...,...,...,...
10995,P999,84,F,White,4/29/2008,02263,CPT,1,NaN,NaN
10996,P999,84,F,White,4/29/2008,02263,CPT,2,NaN,NaN
10997,P999,84,F,White,4/29/2008,00222,CPT,3,C0370256,NaN
10998,P999,84,F,White,4/29/2008,00222,CPT,4,C0370256,NaN


In [44]:
# Write the dataframe into a csv file
mapped_df_SNOMED.to_csv('./data/mapped_df_SNOMED.csv', index = False)